Imports

In [1]:
import os
import pandas as pd
import google.generativeai as genai
import re
import io

from docling.document_converter import DocumentConverter
from pathlib import Path
from dotenv import load_dotenv 

In [2]:
# Gemini Setup
load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel('gemini-2.0-flash')

Pre-processing

In [3]:
# File processing
file_path = Path("Data\India-Ahmedabad.xlsx")  
filename = file_path.stem  

# Split country and port 
country, port = re.split(r"[-_]", filename, 1)
port = port.replace("-", " ").replace("_", " ").title()

<>:2: SyntaxWarning: invalid escape sequence '\I'
<>:2: SyntaxWarning: invalid escape sequence '\I'
C:\Users\kolla\AppData\Local\Temp\ipykernel_7388\339093756.py:2: SyntaxWarning: invalid escape sequence '\I'
  file_path = Path("Data\India-Ahmedabad.xlsx")


In [4]:
print("country :",country)
print("\nPort :",port)

country : India

Port : Ahmedabad


In [5]:
# Convert Excel to Markdown
converter = DocumentConverter()
result = converter.convert(file_path)
md_data = result.document.export_to_markdown()

md_data

"| Effective: Feb 15, 2024                                           |\n|-------------------------------------------------------------------|\n| DV: Dry Container, High Cube and Non-Operative Reefer             |\n| SP: Special Container (Open Top, Flat Rack, Ventilated, Platform) |\n| RE: Reefer Container                                              |\n\n| All days are calendar or running days                                                                                                                                                                                                                                                     |\n|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|\n| Demurrage: Free time including day of discharge at B/L destination

LLM-Integration

In [6]:
prompt = f"""
Analyze this shipping data and extract the following structured information:

Required Fields:
- S.N (Sequential Number)
- Country: {country}
- Line (ESL)
- Port: {port}
- Equipment Type (e.g: Dry, Reefer, Tank)
- Currency
- Free Days
- Bucket 1 
- Bucket 2 
- Bucket 3 
- Local Charges (if available)

Bucket Rules:
1. Free Days = 0-5 days (no charges)
2. Bucket 1 = 6-10 days
3. Bucket 2 = 11-15 days
4. Bucket 3 = 16-9999 days
Note: Different equipment types may have different free days and bucket ranges - preserve these variations.

Input Data:
{{data}}  # Double curly braces for proper formatting

Return Format (JSON):
{{
  "data": [
    {{
      "S.N": int,
      "Country": "{country}",
      "Line": str,
      "Port": "{port}",
      "Equipment_Type": str, #Format: "20-Dry"
      "Currency": str,
      "Free_days": int,
      "Bucket 1 (6-10)": str, 
      "Bucket 2 (11-15)": str,
      "Bucket 3 (16-9999)": str,
      "Local_Charges": str | null
    }}
  ]
}}

Input data:
{md_data}
"""

In [7]:
# Process with LLM
response = model.generate_content(prompt)
llm_output = response.text

In [8]:
llm_output

'```json\n{\n  "data": [\n    {\n      "S.N": 1,\n      "Country": "India",\n      "Line": "ESL",\n      "Port": "Ahmedabad",\n      "Equipment_Type": "20-Dry",\n      "Currency": "USD",\n      "Free_days": 5,\n      "Bucket 1 (6-10)": "60",\n      "Bucket 2 (11-15)": "82",\n      "Bucket 3 (16-9999)": "110",\n      "Local_Charges": null\n    },\n    {\n      "S.N": 2,\n      "Country": "India",\n      "Line": "ESL",\n      "Port": "Ahmedabad",\n      "Equipment_Type": "40-Dry",\n      "Currency": "USD",\n      "Free_days": 5,\n      "Bucket 1 (6-10)": "120",\n      "Bucket 2 (11-15)": "164",\n      "Bucket 3 (16-9999)": "220",\n      "Local_Charges": null\n    },\n    {\n      "S.N": 3,\n      "Country": "India",\n      "Line": "ESL",\n      "Port": "Ahmedabad",\n      "Equipment_Type": "40-High Cube Reefer",\n      "Currency": "USD",\n      "Free_days": 3,\n      "Bucket 1 (6-10)": "165",\n      "Bucket 2 (11-15)": "220",\n      "Bucket 3 (16-9999)": "275",\n      "Local_Charges": nu

Save Data

In [9]:
# Extract JSON from response
json_str = llm_output[llm_output.find('{'):llm_output.rfind('}')+1]

# Convert to DataFrame and save
df = pd.read_json(io.StringIO(json_str))   
df = pd.json_normalize(df['data'])

# Save with country-port name
df.to_csv(f"{country}-{port}.csv", index=False)
print(f"Saved: {country}-{port}.csv")


Saved: India-Ahmedabad.csv
